In [1]:
import re
import pandas as pd
import PyPDF2, os

### this cell separates the subjects and session of the frame displacement data
abspath = os.path.abspath('__file__')
dname = os.path.dirname(abspath)
os.environ["ROOTDIR"] = dname  # seth path
rootdir = os.environ["ROOTDIR"]

print(rootdir)

c:\Users\angel\Documents\base_dados_disciplinas_ufabc


In [2]:
# Caminho do PDF e do arquivo de saída

pdf_path = os.path.join(rootdir, 'turmas_salas_docentes_2024_3.pdf')
output_path = os.path.join(rootdir, 'turmas_salas_docentes_2024_3.xlsx')

# 1. Ler o PDF
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    number_of_pages = len(reader.pages)

    # Extraímos o texto a partir da segunda página, ignorando a primeira
    text = ""
    for page_num in range(1, number_of_pages):  # Começar da página 2
        text += reader.pages[page_num].extract_text()

# 2. Remover todos os "\n" e adicionar quebras de linha específicas
# Remover todos os "\n" para tratar o texto como uma única linha
text = text.replace("\n", " ")

# Adicionar quebras de linha onde começam novos cursos (exemplo: "BACHARELADO EM" e "LICENCIATURA EM")
text = text.replace("BACHARELADO EM", "\nBACHARELADO EM")
text = text.replace("LICENCIATURA EM", "\nLICENCIATURA EM")

# 2.1 Dividir o texto em linhas novamente após a correção
lines = text.split("\n")

# 3. Inicializar as listas de dados que vão compor as colunas
cursos = []
codigos_turma = []
turmas = []
teorias = []
praticas = []
campus = []
docentes_teoria = []
docentes_teoria_2 = []
docentes_pratica = []
docentes_pratica_2 = []

# 4. Iterar sobre as linhas e preencher as listas com base na estrutura do PDF
for i in range(1, len(lines)):  # Supomos que cada bloco de informações tem 1 linha
    try:
        cursos.append(lines[i].strip())  # Curso
        codigos_turma.append(lines[i + 1].strip())  # Código de Turma
        turmas.append(lines[i + 2].strip())  # Turma
        teorias.append(lines[i + 3].strip())  # Teoria
        praticas.append(lines[i + 4].strip())  # Prática
        campus.append(lines[i + 5].strip())  # Campus
        docentes_teoria.append(lines[i + 6].strip())  # Docente Teoria
        docentes_teoria_2.append(lines[i + 7].strip())  # Docente Teoria 2
        docentes_pratica.append(lines[i + 8].strip())  # Docente Prática
        docentes_pratica_2.append(lines[i + 9].strip())  # Docente Prática 2
    except IndexError:
        # Caso falte alguma linha no bloco, o loop é interrompido (isso pode acontecer em alguns PDFs)
        print(f"Erro ao processar bloco de informações na linha {i}. Dados incompletos.")

# 5. Criar o DataFrame com as colunas organizadas
df = pd.DataFrame({
    "CURSO": cursos,
    "CÓDIGO DE TURMA": codigos_turma,
    "TURMA": turmas,
    "TEORIA": teorias,
    "PRÁTICA": praticas,
    "CAMPUS": campus,
    "DOCENTE TEORIA": docentes_teoria,
    "DOCENTE TEORIA 2": docentes_teoria_2,
    "DOCENTE PRÁTICA": docentes_pratica,
    "DOCENTE PRÁTICA 2": docentes_pratica_2
})

# 6. Salvar o DataFrame como Excel
df.to_excel(output_path, index=False)
print(f"Arquivo Excel gerado com sucesso: {output_path}")

Erro ao processar bloco de informações na linha 898. Dados incompletos.
Erro ao processar bloco de informações na linha 899. Dados incompletos.
Erro ao processar bloco de informações na linha 900. Dados incompletos.
Erro ao processar bloco de informações na linha 901. Dados incompletos.
Erro ao processar bloco de informações na linha 902. Dados incompletos.
Erro ao processar bloco de informações na linha 903. Dados incompletos.
Erro ao processar bloco de informações na linha 904. Dados incompletos.
Erro ao processar bloco de informações na linha 905. Dados incompletos.
Erro ao processar bloco de informações na linha 906. Dados incompletos.


ValueError: All arrays must be of the same length

In [13]:
text

'TURMAS, SALAS E DOCENTES - 2024.307/10/2024- 16:17 CURSO CÓDIGO DE TURMA TURMA TEORIA PRÁTICA CAMPUS DOCENTE TEORIA DOCENTE TEORIA 2 DOCENTE PRÁTICA DOCENTE PRÁTICA 2 \nBACHARELADO EM  BIOTECNOLOGIADANHZ1009-15SABIOLOGIA MOLECULAR E BIOTECNOLOGIA A- Matutino (SA)quarta das 10:00 às 13:00, sala 311-3,  semanal 0 SA FABIANA MEDEIROS DA SILVA \nBACHARELADO EM  BIOTECNOLOGIANANHZ1009-15SABIOLOGIA MOLECULAR E BIOTECNOLOGIA A- Noturno (SA)quarta das 18:00 às 21:00, sala 308-3,  semanal 0 SA FABIANA MEDEIROS DA SILVA \nBACHARELADO EM  BIOTECNOLOGIADANHBT001-23SA BIOLOGIA SINTÉTICA A-Matutino (SA)quinta das 10:00 às 12:00, sala S-304-2,  semanal terça das 08:00 às 10:00, sala 404-3,  semanal SAMILCA RACHEL DA COSTA  RIBEIRO LINSMILCA RACHEL DA COSTA  RIBEIRO LINS \nBACHARELADO EM  BIOTECNOLOGIANANHBT001-23SA BIOLOGIA SINTÉTICA A-Noturno (SA)quinta das 21:00 às 23:00, sala S-304-2,  semanal terça das 19:00 às 21:00, sala 404-3,  semanal SAMILCA RACHEL DA COSTA  RIBEIRO LINSMILCA RACHEL DA COST

In [16]:
import pdfplumber
import pandas as pd

# Função para processar o PDF com pdfplumber e salvar em Excel
# 1. Abrir o PDF com pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    # Inicializar uma lista para armazenar os dados extraídos
    data = []

    # 2. Ler o PDF a partir da segunda página
    for page_num in range(1, len(pdf.pages)):  # Começar da página 2
        page = pdf.pages[page_num]
        text = page.extract_text()

        # 3. Processar o texto linha por linha
        lines = text.split("\n")
        for line in lines:
            # Dividir as colunas da linha e adicionar aos dados
            data.append(line.split())  # Ajuste aqui conforme a formatação do PDF

# 4. Criar um DataFrame com as colunas correspondentes
df = pd.DataFrame(data, columns=["CURSO", "CÓDIGO DE TURMA", "TURMA", "TEORIA", "PRÁTICA", "CAMPUS", 
                                    "DOCENTE TEORIA", "DOCENTE TEORIA 2", "DOCENTE PRÁTICA", "DOCENTE PRÁTICA 2"])

# 5. Salvar em Excel
df.to_excel(output_path, index=False)
print(f"Arquivo Excel gerado com sucesso: {output_path}")


ValueError: 10 columns passed, passed data had 30 columns